HARVARD'S PROJECT - 1

In [10]:
import requests
import pandas as pd
import time

In [ ]:
API_KEY = '49dcb8ef-cc75-4e72-b607-4d4873b0572f'
url = "https://api.harvardartmuseums.org/classification"
params = {
    "apikey": API_KEY,
    "classification": "Paintings",  
    "size": 100,  
    "page": 1
}

response = requests.get(url, params)
data = response.json()

print(data['records'])  

In [ ]:
record=data['records']
for record in data['records']:
    if record['objectcount']>=2500:
        print(record['name'])

API INTEGRATION & BASIC DATA FETCHING:

In [ ]:
#API integration & basic data fetching for one classificatio -> "Painting"

API_KEY = "49dcb8ef-cc75-4e72-b607-4d4873b0572f"                        #This stores the API key
BASE_URL = "https://api.harvardartmuseums.org/object"                   #This base URL is the address of Harvard’s API for object data, where we send our requests.

classification = "Paintings"                                            #this is  choosing a classification type ("Paintings") of artifacts to fetch #You can change this to another classification if needed #You can later allow this to be selected from a dropdown menu in Streamlit.
all_records = []                                                        #all_records: an empty list to store all the artifact results.
records_needed = 2500                                                   #records_needed = 2500: how many artworks you want to collect.
page = 1                                                                #page = 1: we’ll start from the first page of results.

while len(all_records) < records_needed:                                #This loop will continue fetching pages of data until you get all 2500+ records.
    params = {                                                          #These are parameters that you send to the API.
        "apikey": API_KEY,                                              #apikey: your unique key to access the API.
        "classification": classification,                               #classification: filter results (e.g., Paintings only).
        "size": 100,                                                    #size: get 100 records at a time.   # Maximum allowed per call
        "page": page,                                                   #page: which page to load.
        "hasimage": 1                                                   #hasimage = 1: only include items that have an image attached.        # Only objects with images
    }
    print(f"Fetching page {page}...")                                   #This tells you in the terminal which page the script is currently fetching. Just for tracking.

    response = requests.get(BASE_URL, params=params)                    #Sends the request to Harvard’s server and gets the data.
    data = response.json()                                              #.json() converts the server’s response into a usable dictionary in Python.

     # Add the current page's records to our collection                 
    all_records.extend(data["records"])                                 #The API returns one page of 100 records in data["records"]  #extend() adds these to your full all_records list.

    # If there's no next page, stop the loop
    if data["info"]["next"] is None:                                    #Checks if this is the last page of data.
        break                                                           #If there's no next page, the loop stops.

    page += 1                                                           #Goes to the next page (page = 2, then 3, etc.)
    time.sleep(1)                                                       #Waits 1 second before next request to avoid spamming the API.

print(f"\n✅ Collected {len(all_records)} records.")                   #Shows the total number of records collected after finishing the loop.

PYTHON CODE FOR EXTRACTING AND ORGANIZING:

In [ ]:
# METADATA TABLE: PRD-compliant version

metadata_rows = []                                     #Creates an empty list called metadata_rows that will store the information (rows) for each artifact's metadata.

for record in all_records:                             #Loops through every artifact record you've collected in all_records.    
    metadata_rows.append({                             #Makes a new dictionary for each artifact, with keys like id, title, etc.  #Adds (appends) this dictionary to the metadata_rows list.
        'id': record.get('id'),                        # primary key
        'title': record.get('title'),
        'culture': record.get('culture'),
        'period': record.get('period'),
        'century': record.get('century'),
        'medium': record.get('medium'),               #Uses .get('...') to safely extract each field from the record (returns None if the field is missing).
        'dimensions': record.get('dimensions'),
        'department': record.get('department'),
        'description': record.get('description'),
        'classification': record.get('classification'),
        'accessionyear': record.get('accessionyear'),
        'accessionmethod': record.get('accessionmethod'),
    })

# DataFrame and preview
df_metadata = pd.DataFrame(metadata_rows)               #Creates a pandas DataFrame (like an Excel table) called df_metadata using the list of dictionaries you just built.
print(df_metadata.head())                               #Shows the first 5 rows of your metadata table in the output—so you can quickly check your data.

# (Save to CSV for inseration use)
df_metadata.to_csv('artifact_metadata.csv', index=False)

In [ ]:
 # MEDIA TABLE: PRD-compliant version (final)
 
# all_records = your full list of artifact records (dicts) from the API, same as metadata_table source

media_rows = []                                                                                 #Creates an empty list called media_rows that will store the information (rows) for each artifact's metadata.

for record in all_records:                                                                      #Loops through every artifact record you've collected in all_records     
    # Always use this record's ID from metadata as the media-table objectid
    objectid = record.get('id')
    
    # Handle images/media/colors fields; if missing, default to empty list
    images = record.get('images', []) if record.get('images') is not None else []
    media = record.get('media', []) if record.get('media') is not None else []
    colors = record.get('colors', []) if record.get('colors') is not None else []
    
    # Count them (default to 0 if list is empty)
    imagecount = len(images)
    mediacount = len(media)
    colorcount = len(colors)
    
    media_rows.append({
        'objectid': objectid,                  # FK to artifact_metadata.id
        'imagecount': imagecount,
        'mediacount': mediacount,
        'colorcount': colorcount,
        'rank': record.get('rank'),
        'datebegin': record.get('datebegin'),                                                  #Uses .get('...') to safely extract each field from the record (returns None if the field is missing).                   
        'dateend': record.get('dateend'),
    })

# Create DataFrame
df_media = pd.DataFrame(media_rows)                                                            #Creates a pandas DataFrame (like an Excel table) called df_media using the list of dictionaries you just built
print(df_media.head())                                                                         #Shows the first 5 rows of your metadata table in the output—so you can quickly check your data.

# (Save to CSV for insertion use)
df_media.to_csv('artifact_media.csv', index=False)

In [ ]:
# COLORS TABLE: PRD-compliant version

# Build a set of valid metadata IDs to enforce FK integrity
metadata_ids = set(df_metadata['id'])   # If df_metadata exists, else rebuild from all_records

color_rows = []
for record in all_records:                                               #Loops through every artifact record you've collected in all_records 
    if 'colors' in record and record.get('id') in metadata_ids:
        for c in record['colors']:
            color_rows.append({
                'objectid': record.get('id'),          # foreign key to artifact_metadata.id
                'color': c.get('color'),
                'spectrum': c.get('spectrum'),
                'hue': c.get('hue'),
                'percent': c.get('percent'),           # float
                'css3': c.get('css3'),                 # CSS3 hex or name
            })

df_colors = pd.DataFrame(color_rows)                                      #Creates a pandas DataFrame (like an Excel table) called df_media using the list of dictionaries you just built
print(df_colors.head())                                                   #Shows the first 5 rows of your metadata table in the output—so you can quickly check your data.

# (Save to CSV for inseration use)
df_colors.to_csv('artifact_colors.csv', index=False)

In [ ]:
#CHECK AND PREVIEW THE DATA:

print(df_metadata.head())               #Shows the first 5 rows of your metadata table in the output—so you can quickly check your data.
print(df_media.head())                  #Displays the first five rows of the df_media DataFrame, just to check that your data looks correct.
print(df_colors.head())                 #Displays the first five rows of your colors DataFrame so you can check your extracted data.

In [ ]:
#SAVE EACH TABLE AS CSV:

df_metadata.to_csv('artifact_metadata.csv', index=False)     
df_media.to_csv('artifact_media.csv', index=False)           
df_colors.to_csv('artifact_colors.csv', index=False)        

UPLOADING THE DATAFRAMES(CSV FILES) INTO TiDB CLOUD MYSQL TABLES:

In [ ]:
#install MySQL pyton library:

!pip install mysql-connector-python

In [25]:
#connect to visual studio python -> TiDB cloud

import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "EnD345nfx9wxmnG.root",
  password = "Q4KKSkNgKxF3JIPn",
)
mycursor = connection.cursor(buffered=True)

In [ ]:
mycursor.execute("create database HARVARD")

In [28]:
mycursor.execute("use HARVARD")

In [ ]:
#create artifacts_metadata table:

mycursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_metadata (
    id INT PRIMARY KEY,
    title TEXT,
    culture TEXT,
    period TEXT,
    century TEXT,
    medium TEXT,
    dimensions TEXT,
    description TEXT,
    department TEXT,
    classification TEXT,
    accessionyear INT,
    accessionmethod TEXT
)
""")
connection.commit()
print("artifact_metadata table created!")

In [ ]:
#create artifacts_media table:

mycursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_media (
    objectid INT,
    imagecount INT,
    mediacount INT,
    colorcount INT,
    `rank` INT,
    datebegin INT,
    dateend INT
)
""")
connection.commit()
print("artifact_media table created!")


In [ ]:
#create artifacts_colors table:

mycursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_colors (
    objectid INT,
    color TEXT,
    spectrum TEXT,
    hue TEXT,
    percent FLOAT,
    css3 TEXT
)
""")
connection.commit()
print("artifact_colors table created!")

THE FETCHED DATA WILL BE INSERTED TO THE CREATED SQL TABLES:

In [ ]:
#The fetched artifacts_metadata was insereted in created sql table:

# --- Read & Clean Data ---
df = pd.read_csv('artifact_metadata.csv')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.drop_duplicates(subset=['id'])   # <--- Only unique IDs


sql = """
    INSERT INTO artifact_metadata
    (id, title, culture, period, century, medium, dimensions, department, description, classification, accessionyear, accessionmethod)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    try:
        mycursor.execute(sql, tuple(row))
    except mysql.connector.Error as err:
        if err.errno == 1062:
            print(f"Duplicate id: {row['id']}, skipping.")
        else:
            print(f"Error inserting row: {row['id']} -> {err}")

connection.commit()
print("Insert complete (duplicates skipped).")

Insert complete (duplicates skipped).


In [ ]:
#The fetched artifacts_media was insereted in created sql table:

# Step 1: Load your CSV into a DataFrame
df = pd.read_csv('artifact_media.csv')

# Step 2: Convert all columns to object type (for safe null handling)
df = df.astype(object)
df = df.where(pd.notnull(df), None)  # Fix NaN problem

# Step 3: Remove duplicate 'objectid' rows (keep first one only)
df = df.drop_duplicates(subset=['objectid'])  # Adjust key if needed

# Step 4: Define your SQL insert statement
sql = """
INSERT INTO artifact_media
(objectid, imagecount, mediacount, colorcount, `rank`, datebegin, dateend)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

# Step 5: Insert each row, skip duplicates, print errors
for _, row in df.iterrows():
    try:
        mycursor.execute(sql, tuple(row))
    except mysql.connector.Error as err:
        if err.errno == 1062:  # Duplicate PK (objectid) error code
            print(f"Duplicate objectid: {row['objectid']}, skipping.")
        else:
            print(f"Error inserting row: {row['objectid']} -> {err}")

connection.commit()
print("Insert finished (duplicates skipped, no NaN error)!")

In [ ]:
#The fetched artifacts_colors was insereted in created sql table:

# Step 1: Load the CSV into a DataFrame
df = pd.read_csv('artifact_colors.csv')

# Step 2: Ensure all columns allow None, replace NaN with None (handles missing values)
df = df.astype(object)
df = df.where(pd.notnull(df), None)

# Step 3: Remove exact duplicate rows (optional but common with colors/colors)
df = df.drop_duplicates()

# Step 4: Prepare your INSERT SQL statement
sql = """
INSERT INTO artifact_colors
(objectid, color, spectrum, hue, percent, css3)
VALUES (%s, %s, %s, %s, %s, %s)
"""

# Step 5: Prepare data as a list of tuples for batch insertion
data = [tuple(row) for _, row in df.iterrows()]

# Step 6: Batch insert for FAST loading
try:
    mycursor.executemany(sql, data)
    connection.commit()
    print(f"Inserted {mycursor.rowcount} rows successfully into artifact_colors!")
except mysql.connector.Error as err:
    print(f"Error during batch insert: {err}")

# Step 7: Clean up
mycursor.close()
connection.close()
print("All done! 🚀")

In [ ]:
query = "SELECT COUNT(*) FROM artifact_metadata"
mycursor.execute(query)
result = mycursor.fetchone()
print("Row count in artifact_metadata:", result[0])

In [ ]:
query = "SELECT COUNT(*) FROM artifact_media"
mycursor.execute(query)
result = mycursor.fetchone()
print("Row count in artifact_media:", result[0])

In [ ]:
query = "SELECT COUNT(*) FROM artifact_colors"
mycursor.execute(query)
result = mycursor.fetchone()
print("Row count in artifact_colors:", result[0])

STREAMLIT UI DESIGN:

In [ ]:
import streamlit as st
import pandas as pd
import mysql.connector

# ---------------------------------------- Database Connection --------------------------------------------------------
def get_connection():
    return mysql.connector.connect(
        host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
        port=4000,
        user='EnD345nfx9wxmnG.root',
        password='Q4KKSkNgKxF3JIPn',
        database='HARVARD'
    )

# ------------------------------------------ Start Streamlit UI --------------------------------------------------------

st.set_page_config(layout="wide")
st.markdown("<h3 style='color: black;'>🖼️ Harvard Artifacts Collections</h3>", unsafe_allow_html=True)


# ------------------------------------------ App Navigation Menu -------------------------------------------------------

menu = st.radio("Choose a mode:", ["Classification Search", "SQL Queries"], horizontal=True)

conn = get_connection()
cur = conn.cursor()

if menu == "Classification Search":
    st.subheader("🔎 Search by Classification (Ex: Coins, Paintings, ...)")
    
    #--------------------------------------- Fetch classification options from database ---------------------------------------------------
    
    cur.execute("SELECT DISTINCT classification FROM artifact_metadata WHERE classification IS NOT NULL AND classification <> ''")
    classes = sorted([row[0] for row in cur.fetchall()])
    selected_class = st.selectbox("Select classification", ["(Show all)"] + classes)



    if st.button("Show Artifacts"):
        if selected_class == "(Show all)":
            cur.execute("SELECT * FROM artifact_metadata LIMIT 100")
            st.info("Showing up to 100 results. Filter by classification for more precise results.")
        else:
            cur.execute("SELECT * FROM artifact_metadata WHERE classification = %s", (selected_class,))
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        df = pd.DataFrame(rows, columns=columns)
        if df.empty:
            st.warning("No data found for this classification.")
        else:
            st.dataframe(df)



elif menu == "SQL Queries":
    st.subheader("📄 Run PRD/Custom SQL Queries")
    query_dict = {
        "List all artifacts from 14th century belonging to Japanese culture":
            "SELECT * FROM artifact_metadata WHERE century = '14th century' AND culture = 'Japnaese'",
        "What are the unique cultures represented in the artifacts?":
            "SELECT DISTINCT culture FROM artifact_metadata WHERE culture IS NOT NULL AND culture <> ''",
        "List all artifacts from the British Period":
            "SELECT * FROM artifact_metadata WHERE period = 'Brititsh Period'",
        "List artifact titles ordered by accession year (descending)":
            "SELECT title, accessionyear FROM artifact_metadata ORDER BY accessionyear DESC",
        "How many artifacts are there per department?":
            "SELECT department, COUNT(*) AS artifact_count FROM artifact_metadata GROUP BY department ORDER BY artifact_count DESC",
        "Which artifacts have more than 1 image?":
            "SELECT m.id, m.title,a.imagecount FROM artifact_metadata m JOIN artifact_media a ON m.id = a.objectid WHERE a.imagecount > 1",
        "What is the average rank of all artifacts?":  
            "SELECT AVG(rank_num) AS average_rank FROM artifact_media",
        "Which artifacts have a higher mediacount than colorcount?":    
            "SELECT m.id, m.title, a.mediacount, a.colorcount FROM artifact_metadata m JOIN artifact_media a ON  m.id = a.objectid WHERE a.colorcount > a.mediacount",
        " List all artifacts created between 1500 and 1600":
            "SELECT * FROM artifact_media WHERE datebegin >= 1500 AND dateend <= 1600",
        "How many artifacts have no media files?":
            "SELECT COUNT(*) AS no_media_count FROM artifact_media WHERE mediacount = 0",
        "What are all the distinct hues used in the dataset?":
            "SELECT DISTINCT hue FROM artifact_colors", 
        "What are the top 5 most used colors by frequency?":
            "SELECT color, COUNT(*) AS frequency FROM artifact_colors GROUP BY color ORDER BY frequency DESC limit 5",
        "What is the average coverage percentage for each hue":
            "SELECT hue,AVG(percent) AS avg_coverage FROM artifact_colors GROUP BY hue ORDER BY avg_coverage DESC",
        "List all colors used for a given artifact ID":
            "SELECT color FROM artifact_colors WHERE objectid = '1429'",
        "What is the total number of color entries in the dataset?":
            "SELECT COUNT(*) AS total_color_entries FROM artifact_colors",
        "List artifact titles and hues for all artifacts belonging to the Byzantine culture":
            "SELECT m.title, c.hue FROM artifact_metadata m JOIN artifact_colors c ON m.id = c.objectid WHERE m.culture = 'Byzantine'",
        "List each artifact title with its associated hues":
            "SELECT m.title, GROUP_CONCAT(DISTINCT c.hue) AS hues FROM artifact_metadata m JOIN artifact_colors c ON m.id = c.objectid GROUP BY m.title",
        "Get artifact titles, cultures, and media ranks where the period is not null":
            "SELECT m.title, m.culture, a.rank FROM artifact_metadata m JOIN artifact_media a ON m.id = objectid WHERE m.period IS NOT NULL",
        "Find artifact titles ranked in the top 10 that include the color hue Grey":    
            "SELECT DISTINCT (m.title),a.rank_num FROM artifact_metadata m JOIN artifact_media a ON m.id = a.objectid JOIN artifact_colors c ON m.id = c.objectid WHERE c.hue = 'Grey' ORDER BY a.rank_num LIMIT 10",       
        "How many artifacts exist per classification, and what is the average media count for each?":
            "SELECT m.classification, COUNT(*) AS artifact_count, AVG(a.mediacount) AS avg_media_count FROM artifact_metadata m JOIN artifact_media a ON m.id = a.objectid GROUP BY m.classification ORDER BY artifact_count DESC",
        "Find all artifacts first accessioned before the year 1900?":
            "SELECT id, title, accessionyear FROM artifact_metadata WHERE accessionyear < 1900 ORDER BY accessionyear",
        "Show all artifact titles created in the 20th century?":
            "SELECT id, title, medium FROM artifact_metadata WHERE medium LIKE '%Gold%'",
        "Count how many artifacts were made by the culture Chinese ?":
            "SELECT COUNT(*) FROM artifact_metadata WHERE culture = 'Chinese'",
        "List all artifacts whose title contains the word portrait (case-insensitive) ?":
            "SELECT id, title FROM artifact_metadata WHERE LOWER(title) LIKE '%portrait%'"                                              
    }
    
    option = st.selectbox(
        "Choose a prewritten SQL query:",
        list(query_dict.keys())
    )
    default_sql = query_dict[option]
    user_sql = st.text_area("SQL to run:", value=default_sql, height=100)


    if st.button("Run Query"):
        try:
            cur.execute(user_sql)
            rows = cur.fetchall()
            columns = [desc[0] for desc in cur.description]
            df = pd.DataFrame(rows, columns=columns)
            if df.empty:
                st.info("No data found for this query.")
            else:
                st.dataframe(df)
        except Exception as e:
            st.error(f"SQL Error: {e}")

cur.close()
conn.close()